# App


## This is for the streamlit interface: trail 1

In [7]:
#imports
from google import genai
from google.genai import types
import wave
from google.colab import userdata

client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

In [8]:
%pip install streamlit google-generativeai==0.6.0

In [9]:
# app.py
import streamlit as st
from google import genai
from google.genai import types
import wave
import os # Import os to handle temporary file removal
import base64 # Import base64 for download button

# Function to save wave file
def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
   with wave.open(filename, "wb") as wf:
      wf.setnchannels(channels)
      wf.setsampwidth(sample_width)
      wf.setframerate(rate)
      wf.writeframes(pcm)

# Function to get modulated response
def mod_response(transcript):
  # Ensure client is initialized
  try:
      client = genai.Client(api_key=st.secrets["GOOGLE_API_KEY"])
  except KeyError:
      st.error("API key not found. Please set the GOOGLE_API_KEY in your Streamlit secrets.")
      return None
  except Exception as e:
      st.error(f"Failed to initialize Google Generative AI client: {e}")
      return None

  try:
      response = client.models.generate_content(
        model="gemini-2.5-flash-preview-tts",
        contents=transcript, # Text to be spoken
        config=types.GenerateContentConfig(
            response_modalities=["AUDIO"],
            speech_config=types.SpeechConfig(
              voice_config=types.VoiceConfig(
                  prebuilt_voice_config=types.PrebuiltVoiceConfig(
                    voice_name='Gacrux',
                  )
              )
            ),
        )
      )

      # Add error handling for NoneType response or missing parts
      if response and response.candidates and response.candidates[0].content and response.candidates[0].content.parts:
          data = response.candidates[0].content.parts[0].inline_data.data
          file_name='out.wav' # file name
          wave_file(file_name, data) # Saves the file to current directory
          return file_name
      else:
          st.error("Failed to generate audio response. Please try again.")
          return None
  except Exception as e:
        st.error(f"An error occurred during audio generation: {e}")
        return None


# Speech-to-speech translation
def speech_to_speech(user_input_file, translation):
  # Ensure client is initialized
  try:
      client = genai.Client(api_key=st.secrets["GOOGLE_API_KEY"])
  except KeyError:
      st.error("API key not found. Please set the GOOGLE_API_KEY in your Streamlit secrets.")
      return
  except Exception as e:
      st.error(f"Failed to initialize Google Generative AI client: {e}")
      return

  temp_audio_path = "temp_audio.wav"
  try:
      # Save the uploaded file to a temporary location
      with open(temp_audio_path, "wb") as f:
          f.write(user_input_file.getvalue())

      # Upload the local file
      myfile = client.files.upload(file=temp_audio_path)

      transcript = client.models.generate_content(
       model="gemini-2.0-flash",
       contents=myfile).text

      transcript_translation = client.models.generate_content(
      model="gemini-2.0-flash",
      contents="In one line, translate " + transcript + "in " + translation).text

      audio_file = mod_response(transcript_translation)
      if audio_file:
          audio_bytes = open(audio_file, 'rb').read()
          st.audio(audio_bytes, format='audio/wav')
          st.download_button(label="Download Audio",
                             data=audio_bytes,
                             file_name="translated_speech.wav",
                             mime="audio/wav")


  except Exception as e:
      st.error(f"An error occurred during speech-to-speech translation: {e}")
  finally:
      # Clean up the temporary file
      if os.path.exists(temp_audio_path):
          os.remove(temp_audio_path)


# Speech-to-text translation
def speech_to_text(user_input_file, translation):
  # Ensure client is initialized
  try:
      client = genai.Client(api_key=st.secrets["GOOGLE_API_KEY"])
  except KeyError:
      st.error("API key not found. Please set the GOOGLE_API_KEY in your Streamlit secrets.")
      return
  except Exception as e:
      st.error(f"Failed to initialize Google Generative AI client: {e}")
      return

  temp_audio_path = "temp_audio.wav"
  try:
      # Save the uploaded file to a temporary location
      with open(temp_audio_path, "wb") as f:
          f.write(user_input_file.getvalue())

      # Upload the local file
      myfile = client.files.upload(file=temp_audio_path)

      transcript = client.models.generate_content(
       model="gemini-2.0-flash",
       contents=myfile).text

      transcript_translation = client.models.generate_content(
      model="gemini-2.0-flash",
      contents="In one line, translate " + transcript + "in " + translation).text

      st.write(transcript_translation)
  except Exception as e:
      st.error(f"An error occurred during speech-to-text translation: {e}")
  finally:
      # Clean up the temporary file
      if os.path.exists(temp_audio_path):
          os.remove(temp_audio_path)

# Text-to-speech
def text_to_speech(user_input):
  # Ensure client is initialized
  try:
      client = genai.Client(api_key=st.secrets["GOOGLE_API_KEY"])
  except KeyError:
      st.error("API key not found. Please set the GOOGLE_API_KEY in your Streamlit secrets.")
      return
  except Exception as e:
      st.error(f"Failed to initialize Google Generative AI client: {e}")
      return

  try:
      transcript = client.models.generate_content(
       model="gemini-2.0-flash",
       contents="In one line say: " + user_input).text

      audio_file = mod_response(transcript)
      if audio_file:
          audio_bytes = open(audio_file, 'rb').read()
          st.audio(audio_bytes, format='audio/wav')
          st.download_button(label="Download Audio",
                             data=audio_bytes,
                             file_name="generated_speech.wav",
                             mime="audio/wav")
  except Exception as e:
      st.error(f"An error occurred during text-to-speech: {e}")

# Text-to-text
def text_to_text(user_input, translation):
  # Ensure client is initialized
  try:
      client = genai.Client(api_key=st.secrets["GOOGLE_API_KEY"])
  except KeyError:
      st.error("API key not found. Please set the GOOGLE_API_KEY in your Streamlit secrets.")
      return
  except Exception as e:
      st.error(f"Failed to initialize Google Generative AI client: {e}")
      return

  try:
      transcript = client.models.generate_content(
       model="gemini-2.0-flash",
       contents=f"In one line translate: {user_input} into {translation}").text
      st.write(transcript)
  except Exception as e:
      st.error(f"An error occurred during text-to-text: {e}")

# Streamlit App Interface
st.title("Translation and Text-to-Speech App")

feature_selection = st.selectbox(
    "Select a feature:",
    ("Speech-to-Speech", "Speech-to-Text", "Text-to-Speech", "Text-to-Text")
)

user_input_text = None
user_input_file = None
translation_language = None
button_label = "Process" # Default button label

if feature_selection == "Speech-to-Speech" or feature_selection == "Speech-to-Text":
    user_input_file = st.file_uploader("Upload an audio file (.wav)", type=["wav"])
    translation_language = st.text_input("Enter target translation language:")
    button_label = "Translate and Generate Speech" if feature_selection == "Speech-to-Speech" else "Translate to Text"

elif feature_selection == "Text-to-Speech":
    user_input_text = st.text_area("Enter text to convert to speech:")
    button_label = "Generate Speech"

elif feature_selection == "Text-to-Text":
    user_input_text = st.text_area("Enter text to translate:")
    translation_language = st.text_input("Enter target translation language:")
    button_label = "Translate Text"

if st.button(button_label):
    if feature_selection == "Speech-to-Speech":
        if user_input_file and translation_language:
             speech_to_speech(user_input_file, translation_language)
        else:
            st.warning("Please upload an audio file and enter a translation language.")
    elif feature_selection == "Speech-to-Text":
        if user_input_file and translation_language:
            speech_to_text(user_input_file, translation_language)
        else:
            st.warning("Please upload an audio file and enter a translation language.")
    elif feature_selection == "Text-to-Speech":
        if user_input_text:
            text_to_speech(user_input_text)
        else:
            st.warning("Please enter text to convert to speech.")
    elif feature_selection == "Text-to-Text":
        if user_input_text and translation_language:
            text_to_text(user_input_text, translation_language)
        else:
            st.warning("Please enter text to translate and a translation language.")

2025-06-12 20:19:46.467 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 20:19:46.471 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 20:19:46.473 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 20:19:46.474 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 20:19:46.475 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 20:19:46.476 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 20:19:46.477 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 20:19:46.478 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [10]:
!npm install localtunnel -q

⠙⠹⠸⠼⠴⠦
up to date, audited 23 packages in 1s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠦

In [11]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com

34.106.167.62


In [12]:
!npx localtunnel --port 8501

⠙⠹your url is: https://gold-cats-bathe.loca.lt
^C
